# Scrape TED Talks Data

## Libraries

In [1]:
# data cleaning
import pandas as pd
import re

# web scraping
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import random

# misc
import pickle  # probably won't need this

## Notes
* **All features** are scraped by default.
    * You can exclude scraping the transcript by setting `exclude_transcript` to 'True'.
* **English** is the default language.
    * You can specify a language using the `lang` param.
    * See full list of language codes [here].
* **All talks** are scraped by default
    * "All talks" refers to talks that are listed [here](https://www.ted.com/participate/translate/our-languages 'TED languages') for the language you specify via the `lang` param.
    * You may pass in a list of urls to scrape using the `urls` param. However, there are a few limitations:
        * TED must have the talks available in the language you specify.
        * Only one language can be provided per scrape call.

More notes:
* TED doesn't always translate all features even if they have it available under a certain language.
    * Ex: Title and 'About Speaker' might be in English while the transcript is translated to French


## Features

| Feature          | Description                                   | Data Type  |
|------------------|-----------------------------------------------|------------|
| talk_id          | Talk identification number provided by TED    | int        |
| title            | Title of the talk                             | string     |
| speakers         | Speakers in the talk (may be multiple)        | dictionary |
| occupations      | *Occupations of the speakers (may be multiple) | dictionary |
| about_speakers   | *Blurb about each speaker (may be multiple)    | dictionary |
| views            | Count of views                                | int        |
| recorded_date    | Date the talk was recorded                    | string     |
| published_date   | Date the talk was published to TED.com        | string     |
| event            | Event or medium in which the talk was given   | string     |
| native_lang      | Language the talk was given in                | string     |
| available_lang   | All available languages for a talk            | list       |
| comments         | Count of comments                             | int        |
| duration         | Duration in %M%S format                       | string     |
| duration_sec     | Duration in seconds                           | int        |
| topic_tags       | Related tags or topics for the talk           | list       |
| talk_description | Description of the talk                       | string     |
| related_talks    | Related talks                                 | dictionary |
| talk_url         | Url of the talk                               | string     |
| transcript       | Full transcript of the talk                   | string     |

*The dictionary key maps to the speaker in ‘speakers’.

## Languages
TED talks have been subtitled in over 100 languages. You can see the most updated list of talks [here](https://www.ted.com/participate/translate/our-languages 'TED languages').

Below is a list of languages available:  

| code       | language              |
|------------|-----------------------|
| af         | Afrikaans             |
| sq         | Albanian              |
| arq        | Algerian Arabic       |
| am         | Amharic               |
| ar         | Arabic                |
| hy         | Armenian              |
| as         | Assamese              |
| ast        | Asturian              |
| az         | Azerbaijani           |
| eu         | Basque                |
| be         | Belarusian            |
| bn         | Bengali               |
| bi         | Bislama               |
| bs         | Bosnian               |
| bg         | Bulgarian             |
| my         | Burmese               |
| ca         | Catalan               |
| ceb        | Cebuano               |
| zh-cn      | Chinese, Simplified   |
| zh-tw      | Chinese, Traditional  |
| zh         | Chinese, Yue          |
| ht         | Creole, Haitian       |
| hr         | Croatian              |
| cs         | Czech                 |
| da         | Danish                |
| nl         | Dutch                 |
| dz         | Dzongkha              |
| en         | English               |
| eo         | Esperanto             |
| et         | Estonian              |
| fil        | Filipino              |
| fi         | Finnish               |
| fr         | French                |
| fr-ca      | French (Canada)       |
| gl         | Galician              |
| ka         | Georgian              |
| de         | German                |
| el         | Greek                 |
| gu         | Gujarati              |
| cnh        | Hakha Chin            |
| ha         | Hausa                 |
| he         | Hebrew                |
| hi         | Hindi                 |
| hu         | Hungarian             |
| hup        | Hupa                  |
| is         | Icelandic             |
| ig         | Igbo                  |
| id         | Indonesian            |
| inh        | Ingush                |
| ga         | Irish                 |
| it         | Italian               |
| ja         | Japanese              |
| kn         | Kannada               |
| kk         | Kazakh                |
| km         | Khmer                 |
| tlh        | Klingon               |
| ko         | Korean                |
| ku         | Kurdish               |
| ky         | Kyrgyz                |
| lo         | Lao                   |
| ltg        | Latgalian             |
| la         | Latin                 |
| lv         | Latvian               |
| lt         | Lithuanian            |
| lb         | Luxembourgish         |
| rup        | Macedo                |
| mk         | Macedonian            |
| mg         | Malagasy              |
| ms         | Malay                 |
| ml         | Malayalam             |
| mt         | Maltese               |
| mr         | Marathi               |
| mfe        | Mauritian Creole      |
| mn         | Mongolian             |
| srp        | Montenegrin           |
| ne         | Nepali                |
| nb         | Norwegian Bokmal      |
| nn         | Norwegian Nynorsk     |
| oc         | Occitan               |
| ps         | Pashto                |
| fa         | Persian               |
| pl         | Polish                |
| pt         | Portuguese            |
| pt-br      | Portuguese, Brazilian |
| pa         | Punjabi               |
| ro         | Romanian              |
| ru         | Russian               |
| ry         | Rusyn                 |
| sc         | Sardinian             |
| sr         | Serbian               |
| sh         | Serbo-Croatian        |
| szl        | Silesian              |
| si         | Sinhala               |
| sk         | Slovak                |
| sl         | Slovenian             |
| so         | Somali                |
| es         | Spanish               |
| sw         | Swahili               |
| sv         | Swedish               |
| art-x-bork | Swedish Chef          |
| tl         | Tagalog               |
| tg         | Tajik                 |
| ta         | Tamil                 |
| tt         | Tatar                 |
| te         | Telugu                |
| th         | Thai                  |
| bo         | Tibetan               |
| aeb        | Tunisian Arabic       |
| tr         | Turkish               |
| tk         | Turkmen               |
| uk         | Ukrainian             |
| ur         | Urdu                  |
| ug         | Uyghur                |
| uz         | Uzbek                 |
| vi         | Vietnamese            |

# Getting the data

In [2]:
"""TedScraper
Created: 2020-04-15
Updated: –
Author: Miguel Corral Jr.
"""

class SoupMaker:
    """HOLDER 'www.ted.com/talks'."""

    def sleep_short(self):
        """Suspends execution time between .5 - 2 seconds."""
        return time.sleep(random.uniform(.5, 2))

    def sleep_long(self):
        """Suspends execution time between 4 - 6 seconds."""
        return time.sleep(random.uniform(4, 6))
    
    def make_soup(self, url):
        """Returns soup object from a URL."""
        # generate random user-agent
        user_agent = {'User-agent': UserAgent().random}
        # request page and make soup
        page = requests.get(url, headers=user_agent)
        soup = BeautifulSoup(page.content, 'lxml')
        return soup


In [9]:
class TalkFeatures(SoupMaker):
    def get_talk_id(self, soup):
        """Returns the talk_id provided by TED."""
        talk_id = re.search(r"(?<=\"current_talk\":)\"(\d+)\"", soup.text).group(1)
        return talk_id

    def get_title(self, soup):
        """Returns the title of the talk."""
        title_tag = soup.find(attrs={'name': 'title'}).attrs['content']
        title = title_tag.split(':')[1].strip()
        return title

    def get_speakers(self, soup):
        """Returns dict of all speakers per talk."""
        speaker_tag = re.findall(r"(?<=\"speakers\":).*?]", soup.text)[0]
        # convert to DataFrame
        speakers_df = pd.read_json(speaker_tag)
        full_name_raw = (speakers_df.loc[:, 'firstname'] + ' '
                     + speakers_df.loc[:, 'middleinitial'] + ' '
                     + speakers_df.loc[:, 'lastname'])
        full_name_clean = full_name_raw.str.replace('\s+', ' ')
        # transform series to a dict
        speakers = full_name_clean.to_dict()
        return speakers

    def get_occupations(self, soup):
        """Returns list of the occupation(s) of the speaker(s) per talk."""
        occupations_tag = re.findall(r"(?<=\"speakers\":).*?]", soup.text)[0]
        # convert json to DataFrame
        occupations_series = pd.read_json(occupations_tag)['description']
        if occupations_series.all():
            # clean and create dict
            occupations = occupations_series.str.lower().str.split(', ')
            occupations = occupations.to_dict()
        else:
            occupations = None
        return occupations

    def get_about_speakers(self, soup):
        """Returns dict with each 'About the Speaker' blurb per talk."""
        speaker_tag = re.findall(r"(?<=\"speakers\":).*?]", soup.text)[0]
        # convert to DataFrame
        about_series = pd.read_json(speaker_tag)['whotheyare']
        if about_series.all():
            # transform series to a dict
            about_speakers = about_series.to_dict()
        else:
            about_speakers = None
        return about_speakers

    def get_views(self, soup):
        """Returns viewed count per talk."""
        view_count = re.search(r"(?<=\"viewed_count\":)\d+", soup.text).group(0)
        return view_count

    def get_recorded_date(self, soup):
        """Returns date a talk was recorded."""
        recorded_at = re.search(r"(?<=\"recorded_at\":)\"(.*?)T", soup.text).group(1)
        return recorded_at

    def get_published_date(self, soup):
        """Returns date a talk was published in TED.com."""
        published_at = soup.find(attrs={'itemprop': 'uploadDate'}).attrs['content']
        return published_at

    def get_event(self, soup):
        """Returns name of the event in which the talk was given."""
        event = re.search(r"(?<=\"event\":)\"(.*?)\"", soup.text).group(1)
        return event
    
    def get_native_lang(self, soup):
        """Returns native language code for each talk as a string."""
        native_lang = re.search(r'(?<=nativeLanguage\":\")[\w-]+', soup.text).group(0)
        return native_lang
    
    def get_available_lang(self, soup):
        """Returns list of all available languages (lang codes) for a talk."""
        languages = re.findall(r'(?<=languageCode\":\")[\w-]+', soup.text)
        clean_lang = sorted(list(set(languages)))
        return clean_lang

    def get_comments_count(self, soup):
        """Return the count of comments per talk."""
        try:
            comments_count = re.search(r"(?<=\"count\":)(\d+)", soup.text).group(1)
        except AttributeError:
            comments_count = None
        return comments_count

    def get_duration(self, soup):
        """Returns duration of a talk (format ex: 12M43S)"""
        duration_tag = soup.find(attrs={'itemprop': 'duration'}).attrs['content']
        duration = duration_tag.split('PT')[1]
        return duration

    def get_duration_sec(self, soup):
        """Returns duration of a talk in seconds."""
        duration =  re.search(r"(?<=\"duration\":)(\d+)", soup.text).group(1)
        return duration

    def get_topic_tags(self, soup):
        """Returns list of tags (topics) per talk."""
        match_obj = re.search(r"\"tag\":\"(.*?)\"", soup.text)
        tags = match_obj.group(1).split(',')
        return tags
    
    def get_related_talks(self, soup):
        """Returns dict (keys: id & title) of related talks."""
        related_tag = re.search(r"(?<=\"related_talks\":).*?]", soup.text).group(0)
        related_series = pd.read_json(related_tag)
        related_talks = related_series.loc[:, ['id', 'title']].to_dict()
        return related_talks

    def get_talk_url(self, soup):
        """Returns url for each talk as a string."""
        talk_tag = soup.find(attrs={'property': 'og:url'}).attrs['content']
        talk_url = talk_tag.split('/transcript')[0]
        return talk_url

    def get_talk_description(self, soup):
        """Returns description of the talk."""
        desc_tag = soup.find(attrs={'property': 'og:description'}).attrs['content']
        talk_desc = desc_tag.split(': ', 1)[1]
        return talk_desc

    def get_transcript(self, soup):
        """Returns talk's transcript as a single string.""" 
        transcript = ''
        transcript_strings = []
        for div in soup.find_all('div', class_="Grid__cell flx-s:1 p-r:4"):
            for p in div.find_all('p'):
                # add every string in the transcript to a list
                transcript_strings.append(" ".join(p.text.split()))
            else:
                # after all strings have been added, create a single transcript string
                transcript = " ".join(transcript_strings)
        return transcript


In [10]:
class TEDscraper(TalkFeatures):
    def __init__(self, urls='all', lang='en', exclude_transcript=False):
        self.lang = lang
        self.urls = urls
        self.exclude = exclude_transcript
        self.ted_dict = {}
        self.dict_id = 0
        self.failed_counter = 0

    def get_languages(self):
        """Returns DataFrame of all language codes supported by TED."""
        lang_url = 'https://www.ted.com/participate/translate/our-languages'
        soup = self.make_soup(lang_url)
        lang_list = []
        lang_tags = soup.find_all('div', class_='h9')
        for tag in lang_tags:
            if tag.a == None:
                continue
            else:
                lang_code = re.search(r'(?<=\=)[\w-]+', tag.a['href']).group(0)
                lang_name = tag.text
                lang_list.append([lang_code] + [lang_name])
        lang_df = pd.DataFrame(data=lang_list, columns=['code', 'language'])
        return lang_df

    def get_max_page(self):
        """Returns max pagination number from www.ted.com/talks."""
        page_num = [1]
        # make soup from ted.com/talks with specified language
        soup = self.make_soup('https://www.ted.com/talks?language='
                              + self.lang + '&page=1&sort=newest')
        # iterate through each pagination element and get the max
        page_elem = soup.find_all('a', class_='pagination__item pagination__link')
        for element in page_elem:
            page_num.append(int(element.text))
        return max(page_num)
    
    def get_all_url_paths(self):
        """Returns list of all the talk url paths available in www.ted.com/talks"""
        url_path_list = []
        # construct url with lang code specified by the user
        talks_url = ('https://www.ted.com/talks?language='
                    + self.lang + '&page=')
        # set range from 1 to the max page in the pagination element
        page_range = range(self.get_max_page())
        # iterate through each page and get the url for each talk
        for i in page_range:
            # try a second attempt if first attempt fails
            for attempt in range(2):
                try:
                    talks_page_url = talks_url + str(i) + '&sort=newest'
                    soup = self.make_soup(talks_page_url)
                    # delay between searches
                    self.sleep_short()
                    for div in soup.find_all('div', attrs={'class': 'media__image'}):
                        for a in div.find_all('a'):
                            url_path_list.append(a.get('href'))
                except:
                    # delay before continuing to second attempt
                    self.sleep_long()
                # break from attempts loop if no exceptions are raised
                else:
                    break
        return url_path_list

    def get_all_urls(self):
        """Returns list of complete urls for each talk's transcript page."""
        url_list = []
        for url in self.get_all_url_paths():
            url_list.append(('https://www.ted.com'
                             + url.replace(
                                 # to replace
                                 '?language=' + self.lang,
                                 # replace with
                                 '/transcript' + '?language=' + self.lang)
                            ))
        return url_list
    
    def clean_urls(self, urls):
        """Returns list of clean urls from urls the user inputs."""
        clean_urls = []
        for idx, url in enumerate(urls):
            if url.startswith('https://www.ted.com/talks'):
                parts = url.split('/')
                joined = '/'.join(parts[:5])
                clean = joined.split('?')
                lang = clean[0] + '/transcript?language=' + self.lang
                clean_urls.append(lang)
            else:
                print(f'bad url @ {idx} >> {url}')
                continue
        return clean_urls
    
    def scrape_all_features(self, soup):
        """Scrapes all features to a nested dict."""
        # create nested dict
        self.ted_dict[self.dict_id] = {}
        nested_dict = self.ted_dict[self.dict_id]
        # add the features to the nested dict
        nested_dict['talk_id'] = self.get_talk_id(soup)
        nested_dict['title'] = self.get_title(soup)
        nested_dict['speakers'] = self.get_speakers(soup)
        nested_dict['occupations'] = self.get_occupations(soup)
        nested_dict['about_speakers'] = self.get_about_speakers(soup)
        nested_dict['views'] = self.get_views(soup)
        nested_dict['recorded_date'] = self.get_recorded_date(soup)
        nested_dict['published_date'] = self.get_published_date(soup)
        nested_dict['event'] = self.get_event(soup)
        nested_dict['native_lang'] = self.get_native_lang(soup)
        nested_dict['available_lang'] = self.get_available_lang(soup)
        nested_dict['comments'] = self.get_comments_count(soup)
        nested_dict['duration'] = self.get_duration(soup)
        nested_dict['duration_sec'] = self.get_duration_sec(soup)
        nested_dict['topic_tags'] = self.get_topic_tags(soup)
        nested_dict['related_talks'] = self.get_related_talks(soup)
        nested_dict['talk_url'] = self.get_talk_url(soup)
        nested_dict['talk_description'] = self.get_talk_description(soup)
        # add transcript if param is set to False (default)
        if not self.exclude:
            nested_dict['transcript'] = self.get_transcript(soup)
        return nested_dict

    def get_data(self):
        """Returns nested dictionary of features from each talk's transcript page."""
        print("Getting all urls...")
        # define url attribute
        if self.urls == 'all':
            urls = self.get_all_urls()
        else:
            if isinstance(self.urls, list):
                urls = self.clean_urls(self.urls)
            else:
                print("'urls' param needs to be a list")
        print(f"Scraping {len(urls)} TED talks in '{self.lang}'...")
#         print(f"Estimated scrape time is {(1.5*len(urls)/60)} minutes\n")
        # iterate through each ted talk transcript url
        for url in urls:
            # make soup
            soup = self.make_soup(url)
            # taste soup
            taster = soup.title.text
            bad_soup = re.search(r'404: Not Found', taster)
            if bad_soup:
                print(f"\nBad soup! TED might not have this talk available in "
                      f"'{self.lang}'. Check the url\n{url}\n")
                self.failed_counter += 1
                continue
            # delay between searches
            self.sleep_short()
            # try up to three attempts to scrape data
            for attempt in range(1, 3+1):
                try:
                    # create nested dict
                    self.ted_dict[self.dict_id] = {}
                    # scrape features and add to a nested dict
                    self.scrape_all_features(soup)
                    # indicate successful scrape
                    print(self.dict_id, url)
                    # add 1 to create a new nested dict
                    self.dict_id += 1
                except Exception as e:
                    # if the last attempt fails, update the failed counter
                    # and print the exception & talk url
                    if attempt == 3:
                        self.failed_counter += 1
                        print(f'position: {self.dict_id}, exception: {e}, url: {url}\n')
                        continue
                    # delay before another attempt
                    self.sleep_long()
                # break if no exceptions are raised
                else:
                    break
        print(f"""\nTed.com scraping results:
            \n\t• Successful: {self.dict_id}
            \n\t• Failed: {self.failed_counter}\n""")
        return self.ted_dict
        return soup
    
    def to_dataframe(self, ted_dict):
        """Creates DataFrame object from dict."""
        df = pd.DataFrame.from_dict(ted_dict, orient='index')
        return df


# Testing

Output types
https://pandas.pydata.org/pandas-docs/stable/reference/frame.html#serialization-io-conversion

In [13]:
# get languages
test = TEDscraper()

In [ ]:
urls = [
    'https://www.ted.com/talks/sarah_kaminsky_my_father_the_forger/',
    'https://www.ted.com/talks/jorge_drexler_poetry_music_and_identity/transcript',
    'https://www.ted.com/talks/sir_ken_robinson_do_schools_kill_creativity/',
    'https://www.ted.com/talks/paul_mceuen_and_marc_miskin_tiny_robots_with_giant_potential/transcript',
    'https://www.ted.com/talks/antara_raychaudhuri_and_iseult_gillespie_the_legend_of_annapurna_hindu_goddess_of_nourishment/',
    'https://www.ted.com/talks/diana_reiss_peter_gabriel_neil_gershenfeld_and_vint_cerf_the_interspecies_internet_an_idea_in_progress/',
]

In [ ]:
url = ['https://www.ted.com/talks/diana_reiss_peter_gabriel_neil_gershenfeld_and_vint_cerf_the_interspecies_internet_an_idea_in_progress/']


In [ ]:
get_speakers_2(soup)

In [ ]:
test = TEDscraper(urls=url)
test.get_data()

### Troubleshooting
* check if len speakers is len speaker occupations
    * failed: 'https://www.ted.com/talks/adam_kucharski_how_can_we_control_the_coronavirus_pandemic/transcript?language=ru'
* what if language is NOT available?

## Originals

In [ ]:
def make_soup(url):
    """Make soup for each Ted Talk transcript url."""
    # generate random user-agent
    user_agent = {'User-agent': UserAgent().random}
    
    # request page and make soup
    page = requests.get(url, headers=user_agent)
    soup = BeautifulSoup(page.content, 'lxml')
    return soup

In [ ]:
def get_max_page():
    """Return the max pagination number from Ted's home page."""
    page_num = []
    
    # set language to English and sort by newest talks
    soup = make_soup('https://www.ted.com/talks?language=en&page=1&sort=newest')
    
    # iterate through each pagination element and get the max
    for element in soup.find_all('a', class_='pagination__item pagination__link'):
        page_num.append(int(element.text))
    return max(page_num)

In [ ]:
def get_talk_urls():
    """Return all the talk urls in each talks page.
    Filter for talks in English.
    """
    talk_url_list = []
    
    # set language to English
    talks_url = 'https://www.ted.com/talks?language=en&page='
    
    # set range from 1 to the max page in the pagination element
    page_range = range(1, get_max_page()+1)
    
    # iterate through each page and get the url for each talk
    for i in page_range:
        
        # try a second attempt if first attempt fails
        for attempt in range(2):
            try:
                talks_page_url = talks_url + str(i) + '&sort=newest'
                soup = make_soup(talks_page_url)

                # delay between searches
                time.sleep(random.uniform(1, 2))

                for div in soup.find_all('div', attrs={'class': 'media__image'}):
                    for a in div.find_all('a'):
                        talk_url_list.append(a.get('href'))
            except:
                # delay before continuing to second attempt
                time.sleep(random.uniform(2, 3))
            
            # break if no exceptions are raised
            else:
                break
            
    return talk_url_list

In [ ]:
def construct_url():
    """Construct complete url for each talk's transcript page.
    Remove '?language=en' from the end of the url.
    """
    return ['https://www.ted.com' + url.replace('?language=en', '') + '/transcript' for url in get_talk_urls()]

In [ ]:
def get_data():
    """Return nested dictionary with title, speaker, related tags, and transcript from each talk's transcript page."""
    # create empty dict to nest dicts (1 nested dict per talk)
    ted_dict = {}
    
    # 1. used to create nested dicts in 'ted_dict'
    # 2. used as count for successfully scraped pages
    dict_id = 0
    
    # counter for failed scraped pages
    failed_counter = 0
    
    # iterate through each ted talk transcript url
    for url in construct_url():
        # make soup
        soup = make_soup(url)

        # delay between searches
        time.sleep(random.uniform(.5, 2))
        
        # try up to three attempts to scrape data
        for attempt in range(1, 3+1):
            try:    
                # get title
                title_tag = soup.find(attrs={'name': 'title'}).attrs['content']
                title = title_tag.split(':')[1].strip()

                # get speaker
                speaker = soup.find(attrs={'name': 'author'}).attrs['content']

                # get related tags
                match_obj = re.search(r"\"tag\":\"(.*?)\"", soup.text)
                ted_tags = match_obj.group(1).split(',')

                # get talk description
                desc_tag = soup.find(attrs={'property': 'og:description'}).attrs['content']
                # Split description at "TED Talk Subtitles and Transcript:"
                desc_str = desc_tag.split(': ', 1)[1]

                # get transcript
                transcript = ''
                transcript_strings = []
                for div in soup.find_all('div', class_="Grid__cell flx-s:1 p-r:4"):
                    for p in div.find_all('p'):
                        # Add every string in the transcript to a list
                        transcript_strings.append(" ".join(p.text.split()))
                    else:
                        # after all strings have been added, create a single transcript string
                        transcript = " ".join(transcript_strings)

                # add 1 to create a new dict_id and use it to create a nested dict
                dict_id += 1
                ted_dict[dict_id] = {}

                # add the features above to the nested dict
                ted_dict[dict_id]['title'] = title
                ted_dict[dict_id]['speaker'] = speaker
                ted_dict[dict_id]['tags'] = ted_tags
                ted_dict[dict_id]['description'] = desc_str
                ted_dict[dict_id]['transcript'] = transcript
                
                # indicate successfull scrape
                print(dict_id, url)

            except Exception as e:
                # if the last attempt fails, update the failed counter and print the exception & talk url for debugging
                if attempt == 3:
                    failed_counter += 1
                    print(f'position: {dict_id}, exception: {e}, url: {url}\n')
                    continue
                
                # delay before another attempt
                time.sleep(random.uniform(4, 6))
            
            # break if no exceptions are raised
            else:
                break

    print(f"""Ted.com scraping results:
        \n\t• Success: {dict_id}
        \n\t• Failed: {failed_counter}\n""")
    
    return ted_dict

In [ ]:
# call web scraping function
# ted_dict = get_data()
# print(ted_dict)

## Create DataFrame

In [ ]:
# Save pickle
# with open('data/ted_dict_complete.pkl', 'wb') as f:
#     pickle.dump(ted_dict, f)

In [ ]:
# Create DataFrame
df = pd.DataFrame.from_dict(ted_dict, orient='index')

# Pickle DataFrame
df.to_pickle('data/first_df.pkl')

print(f'Shape: {df.shape}')

We were able to successfully scrape 3904 talks (as of February 21, 2020)!  

Only one talk failed as the transcript doesn't load, maybe we should notify TED about this bug...  
https://www.ted.com/talks/marcus_bullock_an_app_that_helps_incarcerated_people_stay_connected_to_their_families/transcript
